In [1]:
import os, sys
import tensorflow as tf
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
from _global.funcs import *
from _global.bbx import *
import tkinter
import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tensorflow.keras import *
import albumentations as A
from PIL import Image
import cv2
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
@tf.function
def detect_fn(input_tensor, model):
    image, shapes = model.preprocess(input_tensor)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)
    return detections

In [3]:
def predict_for_image(img_np, model, draw_bbxs=False):

    input_tensor = tf.convert_to_tensor(np.expand_dims(img_np, 0), dtype=tf.float32)

    prediction = detect_fn(input_tensor, model)

    num_detections = int(prediction.pop('num_detections'))
    prediction = {key: value[0, :num_detections].numpy()
                for key, value in prediction.items()}


    label_map = label_map_util.load_labelmap(LABEL_MAP_PATH)

    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=1, use_display_name=True)

    category_index = label_map_util.create_category_index(categories)   

    prediction['detection_classes'] = prediction['detection_classes'].astype(np.int64) +1

    if not draw_bbxs:
        return prediction

    img_with_detections = img_np.copy()


    viz_utils.visualize_boxes_and_labels_on_image_array(img_with_detections,
                                                        prediction['detection_boxes'],
                                                        prediction['detection_classes'],
                                                        prediction['detection_scores'],
                                                        category_index,
                                                        min_score_thresh=.5,
                                                        agnostic_mode=False,
                                                        use_normalized_coordinates=True
                                                        )

    return img_with_detections


def predict_for_image_with_path(path, model):
    image_np = np.array(Image.open(path))
    
    return predict_for_image(image_np, model)

In [4]:
def draw_bbx(img, bbx, color=(255, 0, 0), thickness=1):
    p1 = int(bbx[0]), int(bbx[1])
    p2 = int(bbx[2]), int(bbx[3])
    
    cv2.rectangle(img, p1, p2,color, thickness)

def draw_bbxs_on_image(img_tensor, bbxs, img_shape=(*IMAGE_SIZES, 3), bbxs_scaled_down=True, bbxs_are_ratio = True, window_name="default"):
    if not bbxs_scaled_down or bbxs_are_ratio:
        if not bbxs_scaled_down:
            x_ratio = img_shape / IMAGE_SIZES[0]
            y_ratio = img_shape / IMAGE_SIZES[1]

        if bbxs_are_ratio:
            x_ratio = IMAGE_SIZES[0]
            y_ratio = IMAGE_SIZES[1]


        for bbx in bbxs:

            bbx[0] *= x_ratio
            bbx[2] *= x_ratio

            bbx[1] *= y_ratio
            bbx[3] *= y_ratio


    img = img_tensor.numpy()

    for bbx in bbxs:
        draw_bbx(img, bbx)

    cv2.imshow(window_name, img)
    cv2.moveWindow(window_name, -1920, 0)
    cv2.setWindowProperty(window_name, cv2.WND_PROP_TOPMOST, 1)

    while cv2.getWindowProperty(window_name, 0) >= 0:
        k = cv2.waitKey(0)
        if k == 27:
            break
        continue
    
    cv2.destroyAllWindows()

In [5]:
def draw_bbxs_from_tfrecord(tfrecord_path, record_number):
    if not isinstance(tfrecord_path, list):
        tfrecord_path = [tfrecord_path]
    
    dataset = tf.data.TFRecordDataset(tfrecord_path)

    for i in range(len(dataset)):
        if i +1 != record_number:
            continue
        
        curr = dataset[i][0]
        image, bbxs = parse_function(curr)

        draw_bbxs_on_image(image, bbxs, 'from_tfrecord')

In [6]:
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [7]:
config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
configs = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
model = model_builder.build(model_config=config['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=model) 
ckpt.restore(os.path.join(MODEL, 'ckpt-12')).expect_partial()

In [8]:
# [67.77223   47.06517   16.923975  25.885923  50.304874  35.963966
#   6.2854676 10.204706  41.274918  30.414097   4.0224295  7.105852
#  37.16802   27.614225   3.1189811  5.787781  33.005714  24.588493
#   2.2111108  4.368665 ]

# [71.112305  49.371758  17.722971  27.159712  52.783493  37.700424
#   6.570285  10.685776  43.320473  31.924595   4.2354403  7.480593
#  38.9796    28.967466   3.2910812  6.0731983 34.608833  25.760878
#   2.3060017  4.58374  ]

In [18]:
path = os.path.join(TEST_IMAGES, "49--Greeting\\49_Greeting_peoplegreeting_49_2.jpg")
path = os.path.join(TRAIN_IMAGES, "1--Handshaking\\1_Handshaking_Handshaking_1_82.jpg")
img_np_with_detections = predict_for_image_with_path(path, model)
# test = tf.squeeze(test, [0])
# test = tf.cast(test, dtype=tf.uint8)
# f_shape = 1
# for shape in test.shape:
#     f_shape *= shape
# plt.imshow(cv2.cvtColor(img_np_with_detections, cv2.COLOR_BGR2RGB))
# plt.show()
   


img = tf.image.encode_jpeg(img_np_with_detections)
tf.io.write_file(os.path.join(INSTALLATION_PATH, "test123443.jpeg"), img)


# test = tf.reshape(test, f_shape)
# test = test.numpy()
# draw_bbxs_on_image(img_tensor, bbxs)

In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
while cap.isOpened():
    ret, frame = cap.read()

    image_np_with_detections = predict_for_image(model,np.array(frame), True)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (width, height)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break


In [9]:
def get_best_bbx(bbxs, scores):
    _max = -1
    index = 0
    for i in range(len(bbxs)):
        if scores[i] > _max:
            _max = scores[i]
            index = i
    return bbxs[index]

In [39]:
def find_faces_and_create_json():
    input_path = os.path.join(INSTALLATION_PATH, "Niklas_Faces")
    output_path = os.path.join(INSTALLATION_PATH, "faces.json")
    imgs = [(os.path.join(input_path, f),cv2.imread(os.path.join(input_path, f))) for f in os.listdir(input_path)]
    shape = imgs[0][1].shape
    bbxs = []
    _json = "[\r\n"
    for i in range(len(imgs)):
        if i > 0:
            _json +=","
        filename, img = imgs[i]
        prediction = predict_for_image(np.array(img), model)
        prediction['detection_boxes'] = prediction['detection_boxes'].tolist()
        for x in prediction['detection_boxes']:
            x[0] *= shape[1]
            x[1] *= shape[0]
            x[2] *= shape[1]
            x[3] *= shape[0]

        best_bbx = get_best_bbx(prediction['detection_boxes'],prediction['detection_scores'])
        best_bbx_json = Bbx(filename,*best_bbx)
        _json += "\r\n" + best_bbx_json.to_json()
    _json += "]"
    with open(output_path,'w') as f:
        f.write(_json)

In [ ]:
def find_and_extract_face(img, model, output_path_with_filename=None):
    shape = img.shape
    prediction = predict_for_image(np.array(img), model)

    best_bbx = get_best_bbx(prediction['detection_boxes'],prediction['detection_scores'])
    best_bbx[0]*= shape[0]
    best_bbx[1] *= shape[1]
    best_bbx[2] *= shape[0]
    best_bbx[3] *= shape[1]

    face = img[int(shape[0] -best_bbx[2]):int(shape[0] -best_bbx[0]), int(best_bbx[1]):int(best_bbx[3])]
    face = cv2.resize(face, (RECOG_IMAGE_SIZES[1], RECOG_IMAGE_SIZES[0]))
    if output_path_with_filename == None:
        return face
    cv2.imwrite(output_path_with_filename,face)

In [10]:
def find_and_extract_faces(input_path, output_path):
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    imgs = [(f ,cv2.imread(os.path.join(input_path, f))) for f in os.listdir(input_path)]
    for i in range(len(imgs)):
        filename, img = imgs[i]
        find_and_extract_face(img, model, os.path.join(output_path, filename))

In [11]:
    input_path = os.path.join(INSTALLATION_PATH, "Kek_Faces")
    output_path = os.path.join(INSTALLATION_PATH, "Kek_Cropped_Faces")

In [12]:
find_and_extract_faces(input_path, output_path)